<a href="https://colab.research.google.com/github/Rijan-Joshi/Fun-Learning/blob/main/ConvNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ConvNet From Scratch with augmentations**

This will contain:
1. image augmentation for increasing the volume of input  
2. visualization of intermediate features and images
3. Fine-tuning the known architecture for the sake of learning

All will be coded in PyTorch




In [4]:
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#Loading the datasets
transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.CIFAR10(
    root = 'data',
    train = True,
    download = True,
    transform = transform
)

test_dataset = datasets.CIFAR10(
    root = 'data',
    train = False,
    download = True,
    transform = transform
)

100%|██████████| 170M/170M [00:06<00:00, 28.2MB/s]


"Since we are using the concept of data augmentation, we will be using less samples of data. Currently, we have 5000 samples for cat and dogs each. But, we will use the dataset with 1000 training images, 500 validation imgages and 1000 test images"

In [6]:
from torch.utils.data import Subset
def create_balanced_subset(dataset, class_indices, start_index, end_index):

  targets = np.array(dataset.targets)
  selected_indices = []

  for cls in class_indices:
    indices = np.where(targets == cls)[0]
    selected_indices.extend(indices[start_index: end_index])

  dataset = Subset(dataset, selected_indices)
  return dataset

train_data = create_balanced_subset(train_dataset, [3,5], 0, 1000)
validation_data = create_balanced_subset(train_dataset, [3,5], 1000, 1500)
test_data = create_balanced_subset(test_dataset, [3,5], 0, 1000)

In [7]:
def check_subset_balance(subset, name="Subset"):
    # This pulls labels from the original dataset using the subset's internal indices
    labels = [subset.dataset.targets[i] for i in subset.indices]

    unique, counts = np.unique(labels, return_counts=True)
    print(f"--- {name} ---")
    print(f"Total samples: {len(subset)}")
    for cls, count in zip(unique, counts):
        print(f"Class {cls}: {count} samples")

check_subset_balance(train_data, "Training Set")
check_subset_balance(validation_data, "Validation Set")

--- Training Set ---
Total samples: 2000
Class 3: 1000 samples
Class 5: 1000 samples
--- Validation Set ---
Total samples: 1000
Class 3: 500 samples
Class 5: 500 samples


In [8]:
#Changing the class 3 -> 0 and 5 -> 1
class DatasetWrapper:
  def __init__(self, subset, mapping = {3 : 0, 5: 1}):
    self.subset = subset
    self.mapping = mapping

  def __getitem__(self, index):
    image, target = self.subset[index]
    return image, self.mapping[target]

  def __len__(self):
    return len(self.subset)

train_ready = DatasetWrapper(train_data)
validation_ready = DatasetWrapper(validation_data)
test_ready = DatasetWrapper(test_data)

In [9]:
#Creating dataloader
from torch.utils.data import DataLoader

batch_size = 64
train_loader = DataLoader(train_ready, batch_size, shuffle = True)
val_loader = DataLoader(validation_ready, batch_size, shuffle = True)
test_loader = DataLoader(test_ready, batch_size, shuffle = True)

In [ ]:
images, labels = next(iter(train_loader))

images = images[:32].numpy()
images = np.transpose(images, (0, 2, 3, 1)) #Channel to the last
label_mappings  = {
    0: "Cat",
    1: "Dog"
}
plt.figure(figsize = (20, 10))
for i in range(len(images)):
  plt.subplot(4, 8, i + 1)
  plt.imshow(images[i])
  plt.xlabel(f"Label: {label_mappings[labels[i].item()]}")
  plt.xticks([])
  plt.yticks([])
plt.show()
#Visualizing the first batch of the images

In [ ]:
images.size()

In [ ]:
import torch
from torch import nn

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")


In [ ]:
from torch.nn.modules import activation
from torchsummary import summary
#Defining the model

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.ConV_stack = nn.Sequential(

        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2),

        nn.Conv2d(in_channels=32, out_channels=64, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2),

        nn.Conv2d(in_channels = 64, out_channels=128, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2),

        nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2),

        nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1),
        nn.ReLU(),

        nn.AdaptiveAvgPool2d(1),
        nn.Flatten(),

        nn.Linear(512, 1),
        nn.Sigmoid()
    )

  def forward(self, X):
    Y = self.ConV_stack(X)
    return Y

In [ ]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else "cpu"
model = NeuralNetwork().to(device)
summary(model, (3, 32, 32))

In [ ]:
#Defining the optimizer and the loss function for the model
optimizer = torch.optim.RMSprop(model.parameters(), lr = 1e-3)
criterion = nn.BCELoss()

In [ ]:
#Defining the training function

def train(dataloader, model, optimizer, loss_fn):
  num_batches = len(dataloader)

  model.train()
  running_loss = 0.0
  correct = 0
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device).float().unsqueeze(1)

    y_pred = model.forward(X)
    loss = loss_fn(y_pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    running_loss += loss.item()
    pred = (y_pred > 0.5).float()
    correct += (pred == y).sum().item()



In [ ]:
#Defining the test function
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  batches = len(dataloader)

  test_loss = 0

  model.eval()
  with torch.no_grad():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device).float().unsqueeze(1)

      y_pred = model.forward(X)
      loss = loss_fn(y, y_pred)

      test_loss += loss.item()

  avg_test_loss = test_loss / batches
